### Load the data.
Naomi Tack

Be sure to set the appropriate folder paths.

In [ ]:
# Path to the Segmented Data
data_folder_prefix_layer = "../Seg_Featured_Data/Seg_Featured_"
data_folder_prefix_metric = "../Seg_Featured_Data_Updated/Seg_Featured_"

In [ ]:
# Import Cell
import matplotlib.pyplot as plt
import numpy as np
import glob, json

In [ ]:
# LayerData

# Load file names and labels for the processed data

with open("data_labels.json", 'r') as json_file:
    label_dict = json.load(json_file)


file_names = list(label_dict.keys())
print(len(file_names))

labels = []
for fl in file_names:
    labels.append(label_dict[fl])
print(len(labels))

files = [data_folder_prefix_layer+x+".npy" for x in file_names]

# files = glob.glob("Seg_Featured_Data/*.npy")
max_len = 250*250+7
num_padded = 0
flattened_test_load = np.zeros((max_len, ))
print(flattened_test_load.shape)

for fl in files:
    test_load = np.load(fl, None, allow_pickle=True)
    y,x,_ = test_load.shape

    # Ignore depth 0
    # Keep all Depth 1
    #  if the x,y is less than 250x50 pad w 0s
    layers_depth = test_load[:,:,1]
    # print(layers_depth.shape)
    if x < 250:
        layers_depth = np.hstack((layers_depth, np.zeros((y, y-x))))
        num_padded +=1
    if y < 250:
        layers_depth = np.hstack((layers_depth, np.zeros((250-y, 250))))
    layers_depth = layers_depth.flatten()
    # print(layers_depth.shape)


    # Get the metrics from the other depths
    vl, ct = np.unique(test_load[:,:,5], return_counts=True)
    if len(ct)>1:
        num_bp = ct[1]
    else:
        num_bp = 0

    # print("conected components (mean, std)", np.mean(test_load[:y//10-1,:x//10-1,2]), np.std(test_load[:y//10-1,:x//10-1,2]))
    # print("orientaiton (mean, std)", np.mean(test_load[:y//10-1,:x//10-1,3]), np.std(test_load[:y//10-1,:x//10-1,3]))
    # print("mean orientaiton (mean, std)", np.mean(test_load[:250//10-1,:250//10-1,4]), np.std(test_load[:250//10-1,:250//10-1,4]))
    # print("num breakpoints",num_bp)
    # print("dist map (mean, std)", np.mean(test_load[:,:,6]), np.std(test_load[:,:,6]))

    data= np.append(layers_depth, [np.mean(test_load[:y//10-1,:x//10-1,2]), np.std(test_load[:y//10-1,:x//10-1,2]),
                                    np.mean(test_load[:y//10-1,:x//10-1,3]), np.std(test_load[:y//10-1,:x//10-1,3]),
                                    num_bp,
                                    np.mean(test_load[:,:,6]), np.std(test_load[:,:,6])])

    # print(data.shape)
    flattened_test_load = np.vstack((flattened_test_load, data))

print(flattened_test_load.shape,flattened_test_load[0].shape, "num padded:", num_padded)

flattened_data = flattened_test_load[1:,:]
flattened_data.shape, len(labels)



In [ ]:
# MetricData

# Load file names and labels for the processed data
with open("../data_labels.json", 'r') as json_file:
    label_dict = json.load(json_file)

data_files = glob.glob(data_folder_prefix_metric +"*")
avalible_files=[]
for fl in data_files:
    avalible_files.append(fl[len(data_folder_prefix_metric):-len(".npy")])

print(len(avalible_files), avalible_files[:5])
# print(avalible_files[0], list(label_dict.keys())[0])


file_names = set.intersection(set(avalible_files), set(list(label_dict.keys())))
print(len(file_names))

labels = []
for fl in file_names:
    labels.append(label_dict[fl])
print(len(labels))

files = [data_folder_prefix_metric+x+".npy" for x in file_names]

ct, values = np.unique(labels, return_counts=True)
print(ct, values, values/values.sum())


max_len = len([0]*576+[0]*576+[0]*576+[0]*250*2+[0]*1) 
flattened_test_load = np.zeros((max_len, ))
print(flattened_test_load.shape)

for fl in files:
    data = np.load(fl, None, allow_pickle=True)
    flattened_test_load = np.vstack((flattened_test_load, data))

print(flattened_test_load.shape)

flattened_data = flattened_test_load[1:,:]
flattened_data.shape, len(labels)
